# Hands On Astropy Tutorial 
**Tutors:** Léa Jouvin

This hands-on tutorial gives an introduction and overview of the [Astropy](http://www.astropy.org/) Python package for Astronomy and was done by Axel Donath that gave the [online course last year](https://escape2020.github.io/school2021/posts/clase16/).

## Preface
We recommend to follow this tutorial by **executing the code cells on your local machine**, along with the tutor. Every sub-topic we will cover in this tutorial will be concluded by a few **exercises with different levels of difficulty** (*easy*, *advanced* and *hard*). The exercises will not be discussed during the course, however we provide a **sample solution** for all exercises in the **solutions folder** you can look at **after the course** and load them directly in the notebook. You can always ask for help on the exercises on the correspoding Slack channel **#d05-fri11-astropy**.   

The estimated time for this tutorial is ~1.5 hours. We have marked some of the sections that deal with more advanced topics as "optional". 

We're happy to receive any **feedback or questions** on the tutorial via mail to *lea.jouvin@cea.fr* or using the 
repository's [issue tracker](https://github.com/escape2020/school2022/issues). 

## What is Astropy?

![](astropy_banner_96.png)

    "The Astropy Project is a community effort to develop a single core package for Astronomy in Python and foster interoperability between Python astronomy packages."


The concept and structure of the package is decribed in more detail in the first [Astropy paper 2013](http://adsabs.harvard.edu/abs/2013A%26A...558A..33A). The development infrastructure
and status of the v2.0 core package is described in the second [Astropy paper 2018](http://adsabs.harvard.edu/abs/2018AJ....156..123A).

The **Astropy package is structured into several submodules** and we will cover (what we consider) the most important of them in the following order:

1. [astropy.units](http://docs.astropy.org/en/stable/units/index.html) and in particular [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html) to do astronomical calculations with units.

2. [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) and in particular the classes [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) and [Angle](http://docs.astropy.org/en/stable/coordinates/angles.html) to handle astronomical sky positions, coordinate systems and coordinate transformations.

3. [astropy.tables](http://docs.astropy.org/en/stable/table/index.html) and the [Table](http://docs.astropy.org/en/stable/api/astropy.table.Table.html) class to handle astronomical data tables.

4. [astropy.io.fits](http://docs.astropy.org/en/stable/io/fits/index.html) to open and write data files in [FITS format](https://fits.gsfc.nasa.gov/fits_documentation.html).

5. (optional) Plotting of astronomical sky images with [astropy.visualization.wcsaxes](http://docs.astropy.org/en/stable/visualization/wcsaxes/)

In addition to the Astropy core package there is an infrastructure of  [Astropy affiliated packages](http://www.astropy.org/affiliated/) dedicated to specific fields of Astronomy or analysis tasks, such as:

- [Astroplan](https://astroplan.readthedocs.io/en/latest/): observation planning
- [Reproject](http://reproject.readthedocs.io/en/stable/): reprojection of sky images
- [Regions](http://astropy-regions.readthedocs.io/en/latest/): handling of sky regions 
- [Photutils](https://photutils.readthedocs.io/en/stable/): source detection and photometry
- [Gammapy](docs.gammapy.org): gamma-ray Astronomy data analysis
- ...and many others, or maybe your package?


## Other Ressources

There are other ressources with Astropy tutorials, we can recommend:

- [Learn.Astropy](http://learn.astropy.org) webpage with a lot of tutorial material.
- [Astropy documentation](http://docs.astropy.org) webpage, with lots of small usage examples.
- [Astropy Tutorials](https://github.com/astropy/astropy-tutorials) repository (same as linked on Learn.Astropy)
- [Astropy workshop](https://github.com/astropy/astropy-workshop) held the AAS meeting 2019. 
- [STAK project](https://stak-notebooks.readthedocs.io/en/latest/) provided by STSci, with tutorial notebooks for typical IRAF analysis tasks. 
- [Webinar on Youtube](https://www.youtube.com/watch?v=YP42k3J08_o&list=PL7kL5D8ITGyV7zeT-oADweFKHsZNh3tKV) provided by Astronomy Data and Computing Services (ADACS). 

As the content that can be covered in this ~1.5 hour tutorial is very limited, we recommend to also **check-out the ressources** listed above after the school.

See https://www.astropy.org/help.html for list of references how to **get help on Astropy**.

If you would like to contribute to Astropy, please start by reading the [contribute page](http://www.astropy.org/contribute.html)
on the website.

Let's start with the setup and check of the notebook:

## 0. Setup

Check package versions. All examples should work with Astropy > 2.0 and Numpy > 1.11

In [ ]:
%matplotlib inline  
import matplotlib.pyplot as plt

In [ ]:
import numpy as np
import astropy
print('numpy:', np.__version__)
print('astropy:', astropy.__version__)

## 1. Units and Quantities

The [astropy.units]() subpackage provides functions and classes to handle physical quantities with units. 

### 1.1. Basics
The recommended way to import the `astropy.units` submodule is: 

In [ ]:
from astropy import units as u

`Quantities` are created by multiplying any number with a unit object:

In [ ]:
distance = 1. * u.lightyear
print(distance)

Or by passing a string to the general `Quantity` object:

In [ ]:
distance = u.Quantity('1 lyr')

Check the availabe units with tab completion on the units module, `u.<TAB>`.

Quantities can be also created using lists and arrays:

In [ ]:
distances = [1, 3, 10] * u.lightyear
print(distances)

distances = np.array([1, 3, 10]) * u.lightyear
print(distances)

In [ ]:
distances.value

The quantity object has a value attribute, which is a plain `numpy.ndarray`:

In [ ]:
type(distances.value)

And a unit, which is represented by a `astropy.units.core.Unit` object:

In [ ]:
distances.unit

In [ ]:
type(distances.unit)

A quantity behaves in many ways just like a `numpy.ndarray` with an attached unit.

In [ ]:
distances * 10

Many numpy functions will work as expected and return again a `Quantity` object:

In [ ]:
np.max(distances)

In [ ]:
np.mean(distances)

But there are exceptions, where the unit handling is not well defined, e.g. in `np.log` arguments have to be dimensionless, such as:

In [ ]:
#np.log(30 * u.MeV) # Will raise an UnitConversionError
np.log(30 * u.MeV / (1 * u.MeV))

Probably the most useful method on the `Quantity` object is the `.to()` method which allows to convert a quantity to different units:

In [ ]:
distance.to('meter')

In [ ]:
distance.to(u.parsec)

Quantities can be combined with any arithmetical expression to derive other quantities, `astropy.units` will propagate
the units correctly:

In [ ]:
speed_of_light = distance / u.year
print(speed_of_light)
print(speed_of_light.to('km/s'))

In [ ]:
print(speed_of_light.to('angstrom/day'))

The most practical way to work with units is: define the input quantities with units, do the computation and forget about the units, convert the final result to the desired units. In most cases there is no need for intermediate unit conversions.

For standardized unit systems such as `'si'` or `'cgs'` there are convenience attributes on the quantity object:

In [ ]:
speed_of_light.si

In [ ]:
speed_of_light.cgs

### 1.2. Equivalencies

In Astronomy and other fields of physics quantities are often measured in more practical units, which are equivalent to the actual physical unit. In `astropy.units` this is handled with the concept of "equivalencies".  

For example consider units to measure spectral quantities such as wavelength, frequency and energy:

In [ ]:
frequency = 3e20 * u.hertz
frequency.to('MeV', equivalencies=u.spectral())

In [ ]:
frequency.to('picometer', equivalencies=u.spectral())

Or for converting temperatures:

In [ ]:
temperature = 25 * u.Celsius

In [ ]:
temperature.to("Kelvin", equivalencies=u.temperature())

For some parallel units like degrees for temperature (Celsus, fahrenheit)

In [ ]:
with u.imperial.enable():
    print(temperature.to("deg_F", equivalencies=u.temperature()))

### 1.3 Miscellaneous

Astropy provides a lot of builtin physical and astronomical constants quantitites in the [astropy.constants](https://docs.astropy.org/en/stable/constants/index.html) submodule:

In [ ]:
from astropy import constants as const

print(const.c.to('km / s'))

Here is a [list of available constants](http://docs.astropy.org/en/stable/constants/#module-astropy.constants).

If you write a function you can make sure the input is given in the right units using the [astropy.units.quantity_input](http://docs.astropy.org/en/stable/api/astropy.units.quantity_input.html#astropy.units.quantity_input) decorator: 

In [ ]:
@u.quantity_input(frequency=u.hertz, temperature=u.K)
def blackbody(frequency, temperature): 
    pre_factor = 2 * (const.h * frequency ** 3) / const.c ** 2
    exponential_factor = 1. / (np.exp((const.h * frequency) / (const.k_B * temperature)) - 1)
    return pre_factor * exponential_factor

### 1.4 Interfacing quantities with third-party code

When writing code that uses quantities we are typically bound to use it everywhere in the code. But often we'd like to interface with functions (e.g. from [SciPy](https://www.scipy.org/) or [Matplotlib](https://matplotlib.org/)) or other third-party code that doesn't know how to handle quantities.

For Matplotlib, Astropy has a builtin support using the [quantity_support()](http://docs.astropy.org/en/stable/api/astropy.visualization.quantity_support.html) context manager:

In [ ]:
from astropy.visualization import quantity_support

plt.figure(figsize=(8, 5))

temperature = 1000 * u.K
frequencies = np.linspace(1E-5, 0.5e15, 1000) * u.hertz

radiance = blackbody(frequency=frequencies, temperature=temperature)

with quantity_support():   
    plt.plot(frequencies, radiance)

But in other cases there is only the option to convert the quantity to well defined units, strip off the unit and re-attach it after the computation:

In [ ]:
emin, emax = [1, 10] * u.TeV
energies = np.logspace(np.log10(emin.value), np.log10(emax.value), 10) * emin.unit
print(energies)

    However the alternative implementation based on `np.geomspace` (very similar to numpy linspace), just works as expected:

In [ ]:
energies = np.geomspace(emin, emax, 10)
print(energies)

### 1.5 Performance Tips

As the main data structure of the Quantity is a standard numpy array, they are suitable for computations with large arrays. Still you have to take care of a few pitfalls.

When creating a Quantity with `data * u.km` or `data * u.Unit()` by default a copy of the data is made:

In [ ]:
values = np.ones((500, 500, 500))

In [ ]:
%%timeit
quantity = values * u.kpc

If you would like to avoid the copy you can use:

In [ ]:
%%timeit
quantity = u.Quantity(values, unit="kpc", copy=False)

In Astropy > 3.1 a new operator was introduced to achieve the same behaviour (you associate an astropy units to an existing value but you don't create a new Quantity object):

In [ ]:
%%timeit
quantity = values << u.Unit("kpc")

To avoid copies when converting to different units and getting the values, you can use *to_value* instead of using`quantity.to("m").value`.

In [ ]:
quantity = u.Quantity(values, unit="kpc", copy=False)
values = quantity.to_value("m")

More details can be found on http://docs.astropy.org/en/stable/units/index.html#performance-tips.

### 1.6 Exercises

- (*easy*) How long does the light travel from the sun to the earth in minutes? How long does the light travel from the Galactic center (assume a distance of 8 kpc) in years? 

In [ ]:
# %load solutions/solution_light_travel.py

- (*advanced*) Define a new unit called `"baro-meter"`, which is eqivalent to 25 cm and use it to measure the height of the empire state building (assume a height of 381 meters). Please read the [Astropy documentation on combining and defining units](http://docs.astropy.org/en/stable/units/combining_and_defining.html) for an example how to do this (For other ways to measure the height of a building using a barometer see [barometer question on Wikipedia](https://en.wikipedia.org/wiki/Barometer_question)...)

In [ ]:
# %load solutions/solution_baro_meter.py

- (*expert*) Find the frequency with the maximum photon rate of the black body spectrum $\mathcal{B}(\nu, T)$ with temperature $T=5000~\mathrm{K}$. For this you can use a numerical optimization routine such as [scipy.optimize.minimize](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.minimize.html).
As the frequency as well as the radiance can cover many orders of magnitude, the stability of the algorithm can be much improved by minimizing the (negative) logarithm of the radiance $-\log{\mathcal{B}}$ as well as minimizing with respect to the logarithm of the frequency by introducing the variable $x = \log{\nu}$. Define a new function based on this x variable and use scipy.optimize to minimize it.

Finally compare the result against the analytical solution from [Wien's displacement law](https://en.wikipedia.org/wiki/Wien%27s_displacement_law). 

In [ ]:
# %load solutions/solution_black_body.py
# flake8: noqa

## 2. Coordinates

With the submodule [astropy.coordinates](http://docs.astropy.org/en/stable/coordinates/) Astropy provides a framework to handle sky positions in various coordinate systems and transformations between them.


### 2.1 Basics
The basic class to handle sky coordinates is [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html):

In [ ]:
from astropy.coordinates import SkyCoord

It can be created by passing a position angle for longitude and latitude and a keyword specifying a coordinate frame:

In [ ]:
position_crab = SkyCoord(83.63 * u.deg,  22.01 * u.deg, frame='icrs')
print(position_crab)

As for `Quantities` the instanciation with `lists`, `arrays` or even `Quantities` also works:

In [ ]:
positions = SkyCoord([345., 234.3] * u.deg,  [-0.1, 0.2] * u.deg, frame='galactic')

Alternatively the angles can be specified as string:

In [ ]:
position_crab = SkyCoord('5h34m31.97s', '22d0m52.10s', frame='icrs')

# or

position_crab = SkyCoord('5:34:31.97', '22:0:52.10',
                         unit=(u.hour, u.deg), frame='icrs')

Where in the first case the unit doesn't have to specified because it is encoded in the string via `'hms'` and `'dms'`.

A very convenient way to get the coordinates of an individual object is qerying the [Sesame](http://cds.u-strasbg.fr/cgi-bin/Sesame) database with `SkyCoord.from_name()`:

In [ ]:
# SkyCoord.from_name('Crab')

To transform the coordinates to a different coordinate system we can use `SkyCoord.transform_to()`:

In [ ]:
pos_gal = position_crab.transform_to('galactic')

For convenience we can also directly use the `.galactic` or `.icrs` attributes:

In [ ]:
position_crab.galactic

In [ ]:
position_crab.icrs

To access the `longitude` and `latitude` angles individually: 

In [ ]:
position_crab.data.lon

In [ ]:
position_crab.data.lat

### 2.2 Measuring distances between positions in the sky
The angular distance between two [SkyCoord](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html) objects, can be found using the [SkyCoord.separation()](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.separation) method:

In [ ]:
#position_saga = SkyCoord.from_name('Sag A*')
position_saga = SkyCoord(0 * u.deg, 0 * u.deg, frame='galactic')

separation=position_crab.separation(position_saga)
separation

Sometimes the "inverse" operation is also useful: compute a new position based on a given offset and position angle:

In [ ]:
position_crab.directional_offset_by(
    separation=1 * u.deg, position_angle=0 * u.deg
)

### 2.3 ALT - AZ coordinates (optional)

In various cirumstances, e.g. for planning observations, it can be usefull to transform a sky coordinate into a position in the horizontal coordinate system given a location on earth and a time

In [ ]:
from astropy.coordinates import EarthLocation, AltAz
from astropy.time import Time

We define a location using [EarthLocation](http://docs.astropy.org/en/stable/api/astropy.coordinates.EarthLocation.html):

In [ ]:
paris = EarthLocation(lat=48.8567 * u.deg, lon=2.3508 * u.deg)
print(paris.geodetic)

And a time using the [Time](http://docs.astropy.org/en/stable/api/astropy.time.Time.html) object:

In [ ]:
now = Time.now()
print(now)

Now we can define a horizontal coordinate system using the [AltAz]([docs.astropy.org/en/stable/api/astropy.coordinates.AltAz.html) class and use it to convert from the sky coordinate:

In [ ]:
altaz = AltAz(obstime=now, location=paris)
crab_altaz = position_crab.transform_to(altaz)
print(crab_altaz)

### 2.4 Exercises

(*easy*) Define the sky coordinate for your favorite astronomical object and find the angular distance to the Crab Nebula as well as the Galactic center.


In [ ]:
# %load solutions/solution_skycoord_easy.py

(*expert*) Make a plot of the height above horizon vs.time for the crab position at the location of Annecy. Mark the time range where it is visible. Would the Crab Nebula be visible tonight?

We give the following for Annecy location: lat=45.8058° and lon=6.5726°

In [ ]:
# %load solutions/solution_crab_visible.py

## 3. Tables

Astropy provides the [Table](http://docs.astropy.org/en/stable/api/astropy.io.votable.tree.Table.html) class in order to handle data tables.

### 3.1 Basics

Table objects can be created as shown in the following

In [ ]:
from astropy.table import Table

In [ ]:
table = Table()

We add columns to the table like we would add entries to a dictionary

In [ ]:
table['Source_Name'] = ['Crab', 'Sag A*', 'Cas A', 'Vela Junior']
table['GLON'] = [184.5575438, 0, 111.74169477, 266.25914205] * u.deg
table['GLAT'] = [-5.78427369, 0, -2.13544151, -1.21985818] * u.deg
table['Source_Class'] = ['pwn', 'unc', 'snr', 'snr']

By executing the following cell, we get a nicely formatted version of the table printed in the notebook:

In [ ]:
table

### 3.2 Accessing rows and columns

We have access to the defined columns. To check which ones are availbe you can use `Table.colnames`:

In [ ]:
table.colnames

And access individual columns just by their name:

In [ ]:
table['GLON']

And also a subset of columns:

In [ ]:
table[['Source_Name', 'GLON']]

Often, it is handy to get the column data as [astropy.units.Quantity](http://docs.astropy.org/en/stable/api/astropy.units.Quantity.html#astropy.units.Quantity) using the `.quantity` property:

In [ ]:
table['GLON'].quantity

Rows can be accessed using numpy indexing:

In [ ]:
table[0:2]

Or by using a boolean mask (boolean numpy array) for indexing:

In [ ]:
selection = table['Source_Name'] == 'Crab'
table[selection]

### 3.3 Reading / Writing tables to disk
Astropy tables can be serialized into many formats. For an overview see [here](http://docs.astropy.org/en/latest/io/unified.html#built-in-table-readers-writers). To write the table in FITS format we can use:

In [ ]:
table.write('data/example.fits', overwrite=True, format='fits')

In [ ]:
Table.read('data/example.fits')

In [ ]:
table.write('data/example.ecsv', overwrite=True, format='ascii.ecsv')

In [ ]:
!cat data/example.ecsv

### 3.4 Indexing and Grouping

The `Table` object implements and `.add_index()` method (You can find more details [here](http://docs.astropy.org/en/stable/table/indexing.html), but not covered in this tutorial.), that allows to define an "index column" to access rows by the value contained in the index column:

In [ ]:
table.add_index(colnames="Source_Name")

Using the defined index column the row can now be accessed in a "Pandas" style, using the `.loc[]` syntax:

In [ ]:
table.loc["Cas A"]

This works for mutiple keys as well:

In [ ]:
table.loc[["Cas A", "Crab"]]

Astropy's table object also supports the possiblity to group the rows by a given key column. The groups will be defined by the unique values contained in the column defined as key:

In [ ]:
table_grouped = table.group_by("Source_Class")

for group in table_grouped.groups:
    print(group, "\n")

Where each `group` is again a `Table` object:

In [ ]:
type(group)

### 3.5. Join and stack tables (Optional)
astropy.table provides convenient methods to join, and horizontally/vertically stack tables.

#### Vertically stack tables

In [ ]:
from astropy.table import vstack
table_1=table_grouped.groups[0]
table_2=table_grouped.groups[1]

In [ ]:
table_1

In [ ]:
table_2

In [ ]:
vstack_table=vstack([table_1, table_2])
vstack_table

#### Horizontally stack tables

In [ ]:
from astropy.table import hstack
table=Table.read('data/example.fits')
table_1=table[0:2]
table_2 = Table({'EXTRA_INFO': ["Amazing", "Beautiful"]})

In [ ]:
table_1

In [ ]:
table_2

In [ ]:
hstack_table=hstack([table_1, table_2], join_type="outer")
hstack_table

#### Join tables

In [ ]:
from astropy.table import join
table=Table.read('data/example.fits')
table_1=table[0:2]
table_2=table[1:4]

In [ ]:
table_1

In [ ]:
table_2

In [ ]:
join_table=join(table_1, table_2)
join_table

### 3.6. Miscellaneous

These are a few other useful operations when working with Astropy tables.

Sort by key:

In [ ]:
table.sort('GLON')

In [ ]:
table

Note that `.sort()` is an in place operation on the table, i.e. changes the actual table.

To remove a specific row by index:

In [ ]:
table.remove_row(0)

Astropy tables also support row-wise iteration in Python loops:

In [ ]:
for row in table:
    print(row['Source_Name'])

Another useful feature for quickly inspecting the data contained in the table is the `.show_in_browser()` method:

In [ ]:
table.show_in_browser(jsviewer=True)

### 3.7 Exercises

- (*easy*) Add columns with the `RA` and `DEC` coordinates of the objects to the example table.

In [ ]:
# %load solutions/solution_add_radec_columns_table.py

- (*advanced *) Load the Fermi 2FHL catalog table (`'data/fermi_2fhl_catalog.fits'`) and find the brighest source (the brightness value is stored in the `'Flux50'` column). What is the common name of the source (column `'ASSOC'`)?
Find all sources within 10 deg from the position of this source.

In [ ]:
# %load solutions/solution_fermi_2fhl.py

- (*expert*) Make an allsky plot (using an 'Aitoff' projection) of all the sources in the Fermi 2FHL catalog grouped by source class (i.e. choose a different marker color for every source class). The class of the sources is stored in the `'CLASS'` column. There are a few possible ways to identify the groups of source class. One of them is to use [Table.group_by()](http://docs.astropy.org/en/stable/api/astropy.table.Table.html#astropy.table.Table.group_by) followed by a loop over the groups (check Astropy documentation!) another way is to use a regular Python `set` to idenfity the unique entries in the `'CLASS'` column and loop over those. You can use `plt.subplot(projection='aitoff')` to instanciate a matplotlib axes with `'Aitoff'` projection. Note that the `'Aitoff'` projection requires angular positions measured in radians.

In [ ]:
# %load solutions/solution_all_sky_plot.py

## 4. FITS Images and WCS

The [flexible image transport system](https://fits.gsfc.nasa.gov/fits_documentation.html) format (FITS) is widely used data format for astronomical images and tables. As example we will use image data of the supernova remnant [Cassiopeia A](https://en.wikipedia.org/wiki/Cassiopeia_A), taken by the [Chandra X-ray observatory](http://chandra.harvard.edu/).

### 4.1 Basics


In [ ]:
from astropy.io import fits

To open the fits file we use `fits.open()` and just specify the filename as an argument:

In [ ]:
hdulist = fits.open('data/casa_0.5-1.5keV.fits.gz')

You get a hdu list that is the list of header data unit stored in the fits file.

We can retrieve some basic information on the  header data unit (HDU) by calling `.info()`:

In [ ]:
hdulist.info()

It contains only one `PrimaryHDU` with data dimensions `(1024, 1024)` of format `float32`. 

To access the hdu we use:

In [ ]:
image_hdu = hdulist['PRIMARY'] 

#or

image_hdu = hdulist[0] 

We can access the data with the `.data` attribute:

In [ ]:
image_hdu.data

It is a plain 2d numpy array. 

We use `plt.imshow()` to visualize it:

In [ ]:
plt.imshow(image_hdu.data, origin='lower', cmap='inferno')
plt.colorbar()

Additional meta information is stored in the `.header` attribute:

In [ ]:
image_hdu.header

We now use the header information to create a world coordinate to pixel coordinate transformation, using the [astropy.wcs.WCS](http://docs.astropy.org/en/stable/api/astropy.wcs.WCS.html#astropy.wcs.WCS) class.

astropy.wcs contains utilities for managing World Coordinate System (WCS) transformations defined in several elaborate [FITS WCS standard conventions](https://fits.gsfc.nasa.gov/fits_wcs.html). These transformations work both forward (from pixel to world) and backward (from world to pixel).

In [ ]:
from astropy.wcs import WCS

In [ ]:
wcs = WCS(image_hdu.header)
print(wcs)

Using the helper methods [SkyCoord.to_pixel()](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.to_pixel) and [SkyCood.from_pixel()](http://docs.astropy.org/en/stable/api/astropy.coordinates.SkyCoord.html#astropy.coordinates.SkyCoord.from_pixel) we can now convert every position in the image to the corresponding sky coordinate:

In [ ]:
SkyCoord.from_pixel(0, 0, wcs)

In [ ]:
# position_casa = SkyCoord.from_name('Cas A')

# or

position_casa = SkyCoord('23h23m27.94s', '+58d48m42.4s', frame='icrs')

position_casa.to_pixel(wcs)

### 4.2 Exercises

- (*easy*) Read in the data from the other energy bands (`'data/casa_1.5-3.0keV.fits.gz'` and `'data/casa_4.0-6.0keV.fits.gz'`) as well, sum up all the data from the three energy bands. Plot the summed data using [plt.imshow()](https://matplotlib.org/devdocs/api/_as_gen/matplotlib.axes.Axes.imshow.html) again.


   
 

In [ ]:
# %load solutions/solution_plot_casa_three_energy_bands.py

- (*advanced*) Cutout the central region of the image using [astropy.nddata.Cutout2D](http://docs.astropy.org/en/stable/api/astropy.nddata.Cutout2D.html) (size of 0.05 deg x 0.05 deg around the position of `'Cas A'`). Find the sky position of the brightest pixel in the cutout data. For this [np.argmax()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argmax.html) and [np.unravel_index()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.unravel_index.html) might be helpful. Make sure you use the updated `.data` and `.wcs` attributes of the `Cutout2D` object to achieve the latter.

In [ ]:
# %load solutions/solution_cutout_casA.py

- (*expert, very hard*) Calculate a radial profile of the image data to estimate the size of the shell of the supernova remnant. For this sum up all the data that is contained in the rings (specified  by `radii_min` and `radii_max`) around the center position of *Cas A*. Choose a total minimum radius of `0 arcmin` and total maximum radius of `5 arcmin` divided in 50 steps. You can use [np.indices()](https://docs.scipy.org/doc/numpy/reference/generated/numpy.indices.html) to get arrays of x and y pixel positions, that can be transformed to sky positions. Make sure you use clever numpy masking and broadcasting to avoid a Python loop over the `radii_min` and `radii_max` arrays (**hint**: [ndarray.reshape(-1, ...)](https://docs.scipy.org/doc/numpy/reference/generated/numpy.reshape.html#numpy.reshape) might be useful to expand the data and radius arrays to the third dimension).

In [ ]:
# %load solutions/solution_radial_profile.py
# flake8: noqa

# open image
image_hdu = fits.open('data/casa_1.5-3.0keV.fits.gz')['PRIMARY']
wcs = WCS(image_hdu.header)
image_data = image_hdu.data

# define position and integration radius
pos_casa = SkyCoord('23h23m27.94s', '+58d48m42.4s', frame='icrs')

# create an array of sky positions for each pixel
yp, xp = np.indices(image_data.shape)
sky_positions = SkyCoord.from_pixel(xp=xp, yp=yp, wcs=wcs)

# calculate separation image
separation = pos_casa.separation(sky_positions)

# initialize radius arrays
radii = np.linspace(0, 5, 50).reshape(-1, 1, 1) * u.arcmin
radii_min, radii_max = radii[:-1], radii[1:]

# mask out the annulus regions
mask = (separation > radii_min) & (separation < radii_max)

data_masked = mask * image_data.reshape(-1, 1024, 1024)
data_summed = data_masked.sum(axis=-1).sum(axis=-1)

radius_center = (radii_min[:, 0, 0] + radii_max[:, 0, 0]) / 2.

r_dr = radius_center * (radii_max - radii_min).squeeze()

with quantity_support():
    plt.plot(radius_center, data_summed / (2 * np.pi * r_dr))
    plt.xlabel('Radius (arcmin)')
    plt.ylabel('Profile Amplitude (A.U.)')

## 5. Plotting of sky images (optional)

The [Matplotlib](https://matplotlib.org/) package is a very popular plotting package for Python. Astropy provides a helper module [astropy.visualization.wcsaxes](http://docs.astropy.org/en/stable/visualization/wcsaxes/) to simplify plotting of sky images with Matplotlib.

To use it we just pass `projection=wcs` to the `plt.subplot()` function:

In [ ]:
ax = plt.subplot(projection=wcs)
ax.imshow(image_hdu.data, cmap='inferno', origin='lower')
ax.set_xlabel('RA (deg)')
ax.set_ylabel('DEC (deg)')

And we get the plot with the x-axis and y-axis in sky coordinates. 

We can add a coordinate grid with `ax.grid()`:

In [ ]:
ax.grid(linewidth=0.3, linestyle='dashed', color='white')
ax.figure

We add a white cross to mark the center position of `'Cas A'`:

In [ ]:
ra = position_casa.icrs.ra.deg
dec = position_casa.icrs.dec.deg
# the get_transform will transform the input from world to pixel coordinates before it is passed to matplotlib.
ax.scatter(ra, dec, transform=ax.get_transform('icrs'), color='white', marker='x')
ax.figure

We can also add geometrical shapes to the image to illustrate regions:

In [ ]:
from matplotlib.patches import Circle
from astropy.wcs.utils import proj_plane_pixel_scales

In [ ]:
# We convert the center position of CasA to pixel coordinated
xy = ax.wcs.world_to_pixel(position_casa)
#xy = position_casa.to_pixel(ax.wcs)
# Define the size of the circle in pixel
pixel_size = proj_plane_pixel_scales(wcs)[0] * u.deg
radius = 2.2 * u.arcmin / pixel_size

In [ ]:
circle = Circle(
    xy=xy,
    radius=radius.to_value(""),
    edgecolor="w",
    facecolor="None"
)
ax.add_patch(circle)
ax.figure

For plotting more complex shapes and handling of regions in general incl. plotting there is an Astropoy affiliated package named [regions](https://astropy-regions.readthedocs.io/en/stable/).

### 5.1 Exercises

- (*easy*) Make a combined plot of the Chandra data for all energy bands side by side (one row, three columns)
and put the energy information in the title of the subplot. You can use e.g. `plt.subplot()` (check the matplotlib documentation).  


In [ ]:
# %load solutions/plot_combined_chandra_image.py

- (*advanced*) Make a combined RGB image of the Chandra data in different energy bands. Therefore you have to
renormalize the data per energy band to the maximum value of respective energy band and combine the data into a 3d array with a shape that is compatible with `plt.imshow()` (check the matplotlib documentation!). You can also try to experiment with the Astropy function [make_lupton_rgb](http://docs.astropy.org/en/stable/api/astropy.visualization.make_lupton_rgb.html#astropy.visualization.make_lupton_rgb). For this you can find a [tutorial in the Astropy documentation](http://docs.astropy.org/en/stable/visualization/lupton_rgb.html).

In [ ]:
# %load solutions/solution_rgb_chandra_image.py

## Acknowledgements
<img src="logo_escape.png" alt="drawing" width="400"/> This event is organized in the framework and with the support of the European Science Cluster of Astronomy & Particle physics ESFRI research infrastructures (ESCAPE), funded by the European Union's Horizon 2020 - Grant N. 824064.